In [1]:
%pip install scipy --upgrade
import math
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.optimize as opt
import numpy as np
import pandas as pd
import statsmodels.api as sm
%pip install pymoo
import pymoo as pymoo

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
#需求分布
demand_probs = [0.01, 0.02, 0.04, 0.06, 0.09, 0.14, 0.18, 0.22, 0.16, 0.06, 0.02]
lead_time_probs = [0.2, 0.6, 0.2]
lead_times = [3, 4, 5]
demand_values = np.arange(len(demand_probs))
demand_values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [3]:
#Holding cost per item per day
h_val=0.6

unit_price=400
unit_cost=250

#Profit margin per TV sold
unit_profit=unit_price-unit_cost

#Cost per delivery from supplier to the dealer
shipping_cost=500

def InvModelsS(x):
  #
  #Set the seed for reproducibility
  np.random.seed(5566)
  R=x[0]
  Q=x[1]
  #The number of simulation runs
  M=5000
  #The number of days
  T=360
  #The experiment will be repeated for
  #m in 1:M repetitions & t in 1:T duration
  #Vector initialization
  #SL: service level
  SL=np.empty(M)
  daily_profit=np.empty(M)
  #
  for m in range(M):
    inv_onhand=np.empty(T+1)
    inv_onhand[0]=Q
    sales=np.zeros(T)
    orderQ=np.zeros(T)
    #Keep track of inventory on-order but not arrived yet
    inv_onorder=np.array([])
    inv_onorder_stamp=np.array([])
    #The number of deliveries from the supplier
    delivery=0
    #The total amount of lost sales
    loss=0
    #The number of stockouts in a repetition
    stockout=0
    #Simulate demand
    d = np.random.choice(demand_values, size=T, p=demand_probs)
    arrival=0
    #print(f"Repetition {m+1}:")
    #print(f"Demand: {d}")
    for t in range(T):
      if t==0:
        #Selling process
        sales[t]=min(inv_onhand[t],d[t])
        #Check stockout
        if (d[t]>sales[t]):
          stockout=stockout+1
          loss=loss+(d[t]-sales[t])
        #Compute inventory position
        inv_position=inv_onhand[0]
        #Ordering mechanism of (s, S)
        if inv_position<=R:
          #Update inventory on-order
          inv_onorder=np.append(inv_onorder,Q)
          #Simulate stochastic lead time
          time_to_arrive=t+np.random.choice(lead_times, p=lead_time_probs)+1
          #Update time stamp for invenory on-order
          inv_onorder_stamp=np.append(inv_onorder_stamp,time_to_arrive)
          #Record order quantity
          orderQ[t]=Q
        #Update inventory on-hand in the end of each day
        inv_onhand[t+1]=inv_onhand[t]-sales[t]
       ###End t==0
      if t>0:
        #Check if any inventory on-order should arrive
        if np.any(inv_onorder_stamp==t):
          #Update the number of deliveries
          delivery=delivery+1
          #Compute the total of arrived inventories
          index=np.where(inv_onorder_stamp==t)
          arrival=np.sum(inv_onorder[index])
          #Update inventory on-hand before starting the day
          inv_onhand[t]=inv_onhand[t]+arrival
          #Removed those just arrived from inventory on-order
          inv_onorder=np.delete(inv_onorder,index)
          inv_onorder_stamp=np.delete(inv_onorder_stamp,index)
        #Record sales
        sales[t]=min(inv_onhand[t],d[t])
        #Check if any stockout takes place
        if d[t]>sales[t]:
          stockout=stockout+1
          loss=loss+d[t]-sales[t]
        #End if
        #Update inventory position
        inv_position=inv_onhand[t]+np.sum(inv_onorder)
        #Ordering mechanism of (s, S) inventory policy
        if (inv_position<=R):
          inv_onorder=np.append(inv_onorder,Q)
          time_to_arrive=t+np.random.choice(lead_times, p=lead_time_probs)+1
          inv_onorder_stamp=np.append(inv_onorder_stamp,time_to_arrive)
          orderQ[t]=Q
        #Update inventory on-hand in the end of the day
        inv_onhand[t+1]=inv_onhand[t]-sales[t]
        ###End t>0
      #print("t=",t,"d=",d[t],"arrival=",arrival,"; inv_onhand=",inv_onhand[t],"; order(t)=",orderQ[t])
      arrival = 0
    ###End t loop
    #Calculate the service level (i.e., in-stock probability)
    SL[m]=(T-stockout)/t
    #Calculate the average daily profit
    daily_profit[m]=(unit_profit*np.sum(sales)-np.sum(h_val*inv_onhand)-shipping_cost*delivery-unit_profit*loss)/T
    #Print simulation progrss
    #if (np.remainder(m,100)==0):
    #  print("m=",m)
  return -1*np.mean(daily_profit)

In [4]:
##Use Pymoo for numerical optimization
## https://pymoo.org/index.html

##Define the optimization problem first
from pymoo.problems.functional import FunctionalProblem

problemInv=FunctionalProblem(n_var=2,objs=InvModelsS,
             xl=np.array([1,10]),xu=np.array([100,200]))
problemInv.evaluate(np.array([45,110]))

array([-826.86042389])

In [5]:
##Nelder-Mead Search
from pymoo.algorithms.soo.nonconvex.nelder import NelderMead
from pymoo.optimize import minimize

resNM=minimize(problemInv,algorithm=NelderMead(),verbose=True)

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |       20 | -7.088547E+02 | -8.256345E+02
     2 |       22 | -8.253787E+02 | -8.256951E+02
     3 |       24 | -8.259399E+02 | -8.264902E+02
     4 |       26 | -8.261526E+02 | -8.264902E+02
     5 |       28 | -8.265580E+02 | -8.269113E+02
     6 |       29 | -8.266684E+02 | -8.269113E+02
     7 |       31 | -8.268387E+02 | -8.270013E+02
     8 |       33 | -8.269518E+02 | -8.270013E+02
     9 |       35 | -8.269875E+02 | -8.270184E+02
    10 |       37 | -8.270406E+02 | -8.271021E+02
    11 |       41 | -8.270235E+02 | -8.271021E+02
    12 |       43 | -8.270470E+02 | -8.271021E+02
    13 |       47 | -8.270196E+02 | -8.271021E+02
    14 |       49 | -8.270511E+02 | -8.271021E+02
    15 |       53 | -8.270375E+02 | -8.271021E+02
    16 |       55 | -8.270736E+02 | -8.271082E+02
    17 |       59 | -8.270419E+02 | -8.271082E+02
    18 |       61 | -8.270744E+02 | -8.271082E+02
    19 |       65 | -8.270704E+02 | -8.271120E+02


In [6]:
print("Best solution found by Nelder-Mead: \nRQ = %s\nBenefit = %s" % (resNM.X, resNM.F))

Best solution found by Nelder-Mead: 
RQ = [ 44.88870571 101.05558113]
Benefit = [-827.12367527]


In [7]:
##Grid Search for optimal (s*, S*)
xval=np.array([(sval,Sval) for sval in range(1,50,3) for Sval in range(80,150,10)])
xval.shape
xval.shape[0]
resgrid=np.empty(xval.shape[0])
xval[0,]

for i in range(xval.shape[0]):
  resgrid[i]=InvModelsS(xval[i,])

In [8]:
id=np.argmin(resgrid)
print(resgrid[id])
print(xval[id])

-826.9488711111112
[ 43 100]


In [9]:
##Hooke-Jeeves Search
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch

resHK=minimize(problemInv,algorithm=PatternSearch(),verbose=True)

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |       20 | -7.418838E+02 | -8.232755E+02
     2 |       24 | -8.242762E+02 | -8.252768E+02
     3 |       29 | -8.252768E+02 | -8.252768E+02
     4 |       32 | -8.259897E+02 | -8.267027E+02
     5 |       37 | -8.267027E+02 | -8.267027E+02
     6 |       40 | -8.267735E+02 | -8.268443E+02
     7 |       45 | -8.268137E+02 | -8.268443E+02
     8 |       49 | -8.269581E+02 | -8.270719E+02
     9 |       54 | -8.270719E+02 | -8.270719E+02
    10 |       57 | -8.270787E+02 | -8.270854E+02
    11 |       62 | -8.270854E+02 | -8.270854E+02
    12 |       65 | -8.271021E+02 | -8.271188E+02
    13 |       70 | -8.271188E+02 | -8.271188E+02
    14 |       73 | -8.271277E+02 | -8.271365E+02
    15 |       78 | -8.271365E+02 | -8.271365E+02
    16 |       82 | -8.271367E+02 | -8.271369E+02
    17 |       86 | -8.271377E+02 | -8.271385E+02
    18 |       90 | -8.270884E+02 | -8.271385E+02
    19 |       94 | -8.271385E+02 | -8.271385E+02


In [10]:
print("Best solution found by Hooke-Jeeves: \nR = %s\nQ = %s" % (resHK.X, resHK.F))

Best solution found by Hooke-Jeeves: 
R = [45.26648873 99.38786261]
Q = [-827.17006483]
